In [1]:
import nltk
import numpy as np
import pandas as pd
import spacy
import gensim
import WebScraper

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt


C:\Users\HVAD\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\HVAD\Anaconda3\lib\site-packages\sklearn\manifold\t_sne.py:420: DeprecationWarning: invalid escape sequence \s
  """


### Latent Dirichlet Allocation (LDA)

LDA assumes that each document is composed of a collection of topics with varying probabilities and that each topic is a collection of words with varying probabilities. 

## Get the review data

In [2]:
inurl1 = "https://www.tripadvisor.co.uk/Attraction_Review-g190384-d6755801-Reviews"
inurl2 = "-The_House_of_Dionysus-Paphos_Paphos_District.html"

ms = WebScraper.WebScraper(site='tripadvisor',url1=inurl1,
                      url2=inurl2,increment_string1="-or",increment_string2="",
                      total_pages=20,increment=10,silent=False)

ms.fullscraper()

review_data = ms.all_reviews

Getting reviews 0/ 20
Diagnostics: Checking if dataframes are of equal size...
Size: 10
Size: 10
Size: 10
Size: 10
Diagnostics complete!
Getting reviews 1/ 20
Diagnostics: Checking if dataframes are of equal size...
Size: 10
Size: 10
Size: 10
Size: 10
Diagnostics complete!
Getting reviews 2/ 20
Diagnostics: Checking if dataframes are of equal size...
Size: 10
Size: 10
Size: 10
Size: 10
Diagnostics complete!
Getting reviews 3/ 20
Diagnostics: Checking if dataframes are of equal size...
Size: 10
Size: 10
Size: 10
Size: 10
Diagnostics complete!
Getting reviews 4/ 20
Diagnostics: Checking if dataframes are of equal size...
Size: 10
Size: 10
Size: 10
Size: 10
Diagnostics complete!
Getting reviews 5/ 20
Diagnostics: Checking if dataframes are of equal size...
Size: 10
Size: 10
Size: 10
Size: 10
Diagnostics complete!
Getting reviews 6/ 20
Diagnostics: Checking if dataframes are of equal size...
Size: 10
Size: 10
Size: 10
Size: 10
Diagnostics complete!
Getting reviews 7/ 20
Diagnostics: Checki

In [3]:
review_data.head()

,Review,title,Rating,date,fullreview
0,This is awesome and is everything books etc sa...,Spectacular,5,Reviewed 3 days ago,This is awesome and is everything books etc sa...
1,"Increadible mosiacs, large site, bargain at on...",A must see if you are in Paphos,5,Reviewed 3 days ago,"Increadible mosiacs, large site, bargain at on..."
2,This site is vast and we took our time visitin...,Simply superb,5,Reviewed 4 days ago,This site is vast and we took our time visitin...
3,Brilliant day out to this archaeological site....,Fab day Out,5,Reviewed 5 days ago,Brilliant day out to this archaeological site....
4,It’s a really beautiful archeological place in...,A trip to the past,5,Reviewed 6 days ago,It’s a really beautiful archeological place in...


## Get the english stopwords

In [4]:
stopwords = nltk.corpus.stopwords.words('english')

## Clean the reviews

We will use gensim.utils.simple_prepocess to create a list of words for each document.

In [5]:
gensim.utils.simple_preprocess("Where's my dog?",deacc = True)

['where', 'my', 'dog']

deacc=True stands for de-accentise the text. Here's an example from the documentation

In [6]:
gensim.utils.deaccent("Šéf chomutovských komunistů dostal poštou bílý prášek")

'Sef chomutovskych komunistu dostal postou bily prasek'

Let's try this out for a particular review

In [7]:
review_data.iloc[0,0]

'This is awesome and is everything books etc say about the place.  Allow around 2 hours to get around everything and the park.  Take plenty of water and wear a hat!'

In [8]:
gensim.utils.simple_preprocess(review_data.iloc[0,0],deacc = True)

['this',
 'is',
 'awesome',
 'and',
 'is',
 'everything',
 'books',
 'etc',
 'say',
 'about',
 'the',
 'place',
 'allow',
 'around',
 'hours',
 'to',
 'get',
 'around',
 'everything',
 'and',
 'the',
 'park',
 'take',
 'plenty',
 'of',
 'water',
 'and',
 'wear',
 'hat']

We can then remove stopwords from this document (notice the removal of 'this', 'is', 'and' and similar words.)

In [9]:
[word for word in gensim.utils.simple_preprocess(review_data.iloc[0,0],deacc = True) if word not in stopwords]

['awesome',
 'everything',
 'books',
 'etc',
 'say',
 'place',
 'allow',
 'around',
 'hours',
 'get',
 'around',
 'everything',
 'park',
 'take',
 'plenty',
 'water',
 'wear',
 'hat']

Create a function to do this

In [10]:
def cleanDocument(x, stopwords):
    return [word for word in gensim.utils.simple_preprocess(x,deacc = True) if word not in stopwords]

In [11]:
cleanDocument(review_data.iloc[0,0],stopwords)

['awesome',
 'everything',
 'books',
 'etc',
 'say',
 'place',
 'allow',
 'around',
 'hours',
 'get',
 'around',
 'everything',
 'park',
 'take',
 'plenty',
 'water',
 'wear',
 'hat']

OK, so far so good. Create a new column for the list representation of the document for each document

In [12]:
review_data['List'] = review_data['fullreview'].apply(lambda x: cleanDocument(x,stopwords))

In [13]:
review_data.head()

,Review,title,Rating,date,fullreview,List
0,This is awesome and is everything books etc sa...,Spectacular,5,Reviewed 3 days ago,This is awesome and is everything books etc sa...,"[awesome, everything, books, etc, say, place, ..."
1,"Increadible mosiacs, large site, bargain at on...",A must see if you are in Paphos,5,Reviewed 3 days ago,"Increadible mosiacs, large site, bargain at on...","[increadible, mosiacs, large, site, bargain, e..."
2,This site is vast and we took our time visitin...,Simply superb,5,Reviewed 4 days ago,This site is vast and we took our time visitin...,"[site, vast, took, time, visiting, exhibits, h..."
3,Brilliant day out to this archaeological site....,Fab day Out,5,Reviewed 5 days ago,Brilliant day out to this archaeological site....,"[brilliant, day, archaeological, site, mosaics..."
4,It’s a really beautiful archeological place in...,A trip to the past,5,Reviewed 6 days ago,It’s a really beautiful archeological place in...,"[really, beautiful, archeological, place, paph..."


Create Bigrams and Trigrams and more. Bigrams are pairs of words that occur together often. Similarly, Trigrams a 3 words frequently occuring together. This can be extended to a larger number of words occuring together.

In [14]:
# Create bigrams
bigrams = gensim.models.Phrases(review_data['List'], min_count=3, threshold=50)
bigrams_Phrases= gensim.models.phrases.Phraser(bigrams)

# Create trigrams
trigrams = gensim.models.Phrases(bigrams_Phrases[list(review_data['List'])], min_count=3, threshold=50) 
trigram_Phrases = gensim.models.phrases.Phraser(trigrams)

In [15]:
print(bigrams_Phrases[review_data['List'][0]])

['awesome', 'everything', 'books', 'etc', 'say', 'place', 'allow', 'around', 'hours', 'get', 'around', 'everything', 'park', 'take', 'plenty', 'water', 'wear', 'hat', 'spectacular']


Here, we don't see any bigrams but the idea is shown below with an example

In [16]:
a = []

a.append('new york is amazing')
a.append("Yeah I know, it's all about new york")
a.append("What about the tower in new york?")
a.append("new york is the place to be apparently")
a.append("Some more words and new york some more words")
a.append("I loved the show")
a.append("specially in new york")

a_list = list(map(lambda x: cleanDocument(x,stopwords),a))
bigram = gensim.models.Phrases(a_list,min_count=1, threshold=1)
bigram_phraser = gensim.models.phrases.Phraser(bigram)
sent = cleanDocument('Is new york the best place or what?',stopwords)
print(bigram_phraser[sent])

['new_york', 'best', 'place']


So, new york is classed as a bigram. The min_count argument specifies the minimum number of times this bigram should appear before it is accepted as a bigram. The threshold determines how difficult it is the be classified as a bigram.

Let's create a function for this.

In [17]:
def createGrams(ls):
    """
    This function expects a list (or series) of lists of words each being a list representation of a document.
    It returns a list of bigrams and a list of Trigrams relevant to the list given.
    """
    # Create bigrams
    bigrams = gensim.models.Phrases(ls, min_count=3, threshold=50)
    bigrams_Phrases= gensim.models.phrases.Phraser(bigrams)

    # Create trigrams
    trigrams = gensim.models.Phrases(bigrams_Phrases[list(ls)], min_count=3, threshold=50) 
    trigram_Phrases = gensim.models.phrases.Phraser(trigrams)
    
    return [bigram_phraser[i] for i in list(ls)],[trigram_Phrases[i] for i in list(ls)]

In [18]:
createGrams(review_data['List'])

([['awesome',
   'everything',
   'books',
   'etc',
   'say',
   'place',
   'allow',
   'around',
   'hours',
   'get',
   'around',
   'everything',
   'park',
   'take',
   'plenty',
   'water',
   'wear',
   'hat',
   'spectacular'],
  ['increadible',
   'mosiacs',
   'large',
   'site',
   'bargain',
   'euro',
   'would',
   'reccoment',
   'anyone',
   'visiting',
   'paphos',
   'also',
   'great',
   'views',
   'along',
   'cost',
   'towards',
   'coral',
   'beach',
   'must',
   'see',
   'paphos'],
  ['site',
   'vast',
   'took',
   'time',
   'visiting',
   'exhibits',
   'hours',
   'excavations',
   'ongoing',
   'lot',
   'see',
   'especially',
   'beautiful',
   'mosaics',
   'best',
   'come',
   'early',
   'get',
   'tour',
   'busses',
   'visiting',
   'simply',
   'superb'],
  ['brilliant',
   'day',
   'archaeological',
   'site',
   'mosaics',
   'super',
   'thorougly',
   'enjoyable',
   'experience',
   'wear',
   'strudy',
   'shoes',
   'fab',
   'day

Now we can utilise the cleanDocument function to write a new function to give us directly the list representation of a document while considering bigrams and trigrams

In [19]:
def cleanAndCreateGrams(ls,stopwords):
    return(createGrams(ls.apply(lambda x: cleanDocument(x,stopwords)))[0])

In [22]:
review_data['GramList'] = cleanAndCreateGrams(review_data['fullreview'],stopwords)

review_data.head()

,Review,title,Rating,date,fullreview,List,GramList
0,This is awesome and is everything books etc sa...,Spectacular,5,Reviewed 3 days ago,This is awesome and is everything books etc sa...,"[awesome, everything, books, etc, say, place, ...","[awesome, everything, books, etc, say, place, ..."
1,"Increadible mosiacs, large site, bargain at on...",A must see if you are in Paphos,5,Reviewed 3 days ago,"Increadible mosiacs, large site, bargain at on...","[increadible, mosiacs, large, site, bargain, e...","[increadible, mosiacs, large, site, bargain, e..."
2,This site is vast and we took our time visitin...,Simply superb,5,Reviewed 4 days ago,This site is vast and we took our time visitin...,"[site, vast, took, time, visiting, exhibits, h...","[site, vast, took, time, visiting, exhibits, h..."
3,Brilliant day out to this archaeological site....,Fab day Out,5,Reviewed 5 days ago,Brilliant day out to this archaeological site....,"[brilliant, day, archaeological, site, mosaics...","[brilliant, day, archaeological, site, mosaics..."
4,It’s a really beautiful archeological place in...,A trip to the past,5,Reviewed 6 days ago,It’s a really beautiful archeological place in...,"[really, beautiful, archeological, place, paph...","[really, beautiful, archeological, place, paph..."


In [24]:
en_spacy = spacy.load('en')

C:\Users\HVAD\Anaconda3\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\Users\HVAD\Anaconda3\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\Users\HVAD\Anaconda3\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\Users\HVAD\Anaconda3\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\Users\HVAD\Anaconda3\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\Users\HVAD\Anaconda3\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs

In [26]:
doc = en_spacy(" ".join(review_data['GramList'][0]))

In [27]:
doc[2].pos_

'NOUN'

In [28]:
# Create Dictionary
id2word = gensim.corpora.Dictionary(review_data['GramList'])

# Create Corpus
texts = review_data['GramList']

In [29]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [30]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('allow', 1),
  ('around', 2),
  ('awesome', 1),
  ('books', 1),
  ('etc', 1),
  ('everything', 2),
  ('get', 1),
  ('hat', 1),
  ('hours', 1),
  ('park', 1),
  ('place', 1),
  ('plenty', 1),
  ('say', 1),
  ('spectacular', 1),
  ('take', 1),
  ('water', 1),
  ('wear', 1)]]

In [38]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

...

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

In [39]:
lda_model

In [40]:
# Print the Keyword in the 10 topics
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.008*"wonderful" + 0.007*"real" + 0.007*"site" + 0.007*"especially" + 0.007*"restored" + 0.007*"visit" + 0.006*"around" + 0.006*"visited" + 0.006*"work" + 0.006*"first"'), (1, '0.027*"mosaics" + 0.022*"site" + 0.014*"ruins" + 0.013*"see" + 0.013*"well" + 0.013*"roman" + 0.012*"interesting" + 0.011*"park" + 0.010*"buildings" + 0.010*"visit"'), (2, '0.021*"mosaics" + 0.019*"see" + 0.017*"history" + 0.016*"must" + 0.016*"good" + 0.016*"site" + 0.016*"well" + 0.016*"place" + 0.016*"visit" + 0.015*"worth"'), (3, '0.039*"mosaics" + 0.028*"visit" + 0.020*"interesting" + 0.018*"paphos" + 0.018*"park" + 0.017*"site" + 0.017*"see" + 0.016*"archaeological" + 0.016*"well" + 0.015*"place"')]


In [41]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=review_data['GramList'], dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)

C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\HVAD\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)


Perplexity:  -6.5683409957213295

Coherence Score:  0.2894762136456679


In [42]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\HVAD\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.110387  0.039072       1        1  42.096073
2      0.056160 -0.112288       2        1  28.439713
1      0.009672  0.080129       3        1  20.772545
0     -0.176220 -0.006913       4        1   8.691667, topic_info=     Category        Freq         Term       Total  loglift  logprob
134   Default   43.000000  interesting   43.000000  30.0000  30.0000
101   Default   23.000000        ruins   23.000000  29.0000  29.0000
30    Default   43.000000       paphos   43.000000  28.0000  28.0000
140   Default   16.000000        roman   16.000000  27.0000  27.0000
175   Default   23.000000         walk   23.000000  26.0000  26.0000
10    Default   43.000000        place   43.000000  25.0000  25.0000
6     Default   25.000000          get   25.000000  24.0000  24.0000
255   Default   11.000000          sun   11.000000  23.0000  23.0000
212   Default   29.000000         good   29.000000  22.0000  22.0000
47    Default  109.000000      mosaics  109.000000  21.0000  21.0000
258   Default   24.000000         area   24.000000  20.0000  20.0000
384   Default    8.000000    wonderful    8.000000  19.0000  19.0000
170   Default   15.000000         lots   15.000000  18.0000  18.0000
437   Default   14.000000    buildings   14.000000  17.0000  17.0000
43    Default    4.000000   especially    4.000000  16.0000  16.0000
493   Default   12.000000          hot   12.000000  15.0000  15.0000
903   Default    4.000000         real    4.000000  14.0000  14.0000
15    Default   13.000000        water   13.000000  13.0000  13.0000
27    Default   14.000000        large   14.000000  12.0000  12.0000
121   Default    7.000000      visited    7.000000  11.0000  11.0000
191   Default    6.000000         seen    6.000000  10.0000  10.0000
306   Default   19.000000  fascinating   19.000000   9.0000   9.0000
713   Default    4.000000         work    4.000000   8.0000   8.0000
278   Default   18.000000        house   18.000000   7.0000   7.0000
331   Default    5.000000         went    5.000000   6.0000   6.0000
117   Default    5.000000         near    5.000000   5.0000   5.0000
190   Default    3.000000     restored    3.000000   4.0000   4.0000
189   Default   13.000000       mosaic   13.000000   3.0000   3.0000
46    Default   10.000000          lot   10.000000   2.0000   2.0000
29    Default   37.000000         must   37.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
1043   Topic4    0.752319       tracks    1.362051   1.8492  -6.1187
1038   Topic4    0.752319         lead    1.362051   1.8492  -6.1187
1042   Topic4    0.752319       states    1.362051   1.8492  -6.1187
1039   Topic4    0.752319         mind    1.362051   1.8492  -6.1187
1041   Topic4    0.752319      ranking    1.362051   1.8492  -6.1187
853    Topic4    0.751864         pity    1.362301   1.8484  -6.1193
854    Topic4    0.751864     pleasant    1.362301   1.8484  -6.1193
849    Topic4    0.751864      carried    1.362301   1.8484  -6.1193
43     Topic4    2.423544   especially    4.670128   1.7868  -4.9489
851    Topic4    1.883939        first    3.678774   1.7736  -5.2007
665    Topic4    1.372177        kings    2.684712   1.7716  -5.5177
384    Topic4    2.859645    wonderful    8.052171   1.4076  -4.7834
713    Topic4    1.901651         work    4.674264   1.5435  -5.1914
698    Topic4    1.263091        small    3.004905   1.5761  -5.6005
946    Topic4    1.253259      picture    3.010582   1.5664  -5.6084
768    Topic4    1.217990         hard    3.086325   1.5130  -5.6369
331    Topic4    1.617409         went    5.301384   1.2557  -5.3533
800    Topic4    0.976184        italy    2.165056   1.6463  -5.8582
121    Topic4    1.983409      visited    7.746689   1.0804  -5.1493
146    Topic4    1.246172         done    3.429546   1.4305  -5.6140
607    Topic4    1.185345   attraction    3.7